<h1>Import</h1>

In [17]:
import re
import copy
import xml.etree.ElementTree as ET
import pickle
from xml.dom import minidom
print("done")

done


<h1>Sentence Class</h1>

In [18]:
class Sentence:
    def __init__(self):
        self.text = ""
        self.proteins = []
        self.interactions = []
        self.reconstructedText = ""
        self.words = []
        self.valid = False

    def show(self):
        print(self.text)
        print(self.proteins)
        if len(self.interactions):
            for i in self.interactions:
                print(i)
        print()
        
        
    def processText(self):
        #self.text = self.text.replace("_/_","_ _")
        #self.text = self.text.replace("_:_","_ _")
        #self.text = self.text.replace("_-_","_ _")
        self.text = re.sub("(\_[^_]_)", "_ _", self.text)#regexp to remove things like _:_
        self.text = self.text.replace("(_","_")
        self.text = self.text.replace("_)","_")
        
    def convertProtein(self):
        for p in range(len(self.proteins)):
            self.proteins[p]="PROTEIN"+self.proteins[p][-1]
        for i in range(len(self.interactions)):
            self.interactions[i][0]="PROTEIN"+self.interactions[i][0][-1]
            self.interactions[i][1]="PROTEIN"+self.interactions[i][1][-1]
            
    
    def checkValidity(self):
        count = 0
        for w in range(len(self.words)):
            if "__" in self.words[w]:
                count = count +1
        
        if count == len(self.proteins):
            self.valid = True
            #print(self.valid)
        
    
    def reconstructText(self):
        self.words = self.text.split(" ")
        self.checkValidity()
        #print(self.words)
        if self.valid:
            p=0
            for w in range(len(self.words)):
                if "__" in self.words[w]:
                    self.words[w] = self.proteins[p]
                    #self.words[w] = "PROTEIN"+str(p)
                    p=p+1
            #print(self.words)
            #print()
        
    def reassembleSentence(self):
        print(" ".join(self.words))
        
    def fullProcess(self):
        self.processText()
        self.convertProtein()
        self.reconstructText()
        

<h1>Extracting and processing the Training Set</h1>

In [19]:
def extractAndProcessTraining(location):
    root = ET.parse(location).getroot()
    sentences = []
    
    for document in root:
        for sentence in document:
            currentSentence = Sentence()
            #print(sentence.attrib["text"])
            currentSentence.text = sentence.attrib["text"]

            for stuff in sentence:
                if stuff.tag == "entity":
                    #print(stuff.attrib["id"])
                    currentSentence.proteins = currentSentence.proteins + [stuff.attrib["id"]]

                if stuff.tag == "pair":
                    #print(stuff.attrib["e1"],stuff.attrib["e2"],stuff.attrib["interaction"])
                    currentSentence.interactions = currentSentence.interactions + [[stuff.attrib["e1"],stuff.attrib["e2"],stuff.attrib["interaction"]]]
            #print()
            sentences = sentences + [currentSentence]

    #originals = copy.deepcopy(sentences)        

    '''
    for s in sentences:
        s.show()
    '''    
    print("Found ",len(sentences)," sentences, now processing...")
    
    for s in sentences:
        s.fullProcess()
        
    valids = [sentence for sentence in sentences if sentence.valid]
    
    print("Found ",len(valids)," valid sentences")
    
    return valids


In [20]:
locations = ['AIMed-train.xml','BioInfer-train.xml','HPRD50-train.xml','IEPA-train.xml','LLL-train.xml']

trainSet = []

for l in locations:
    trainSet = trainSet + extractAndProcessTraining("Datasets/train/"+l)
    
print("In total we have",len(trainSet),"training examples")

Found  1554  sentences, now processing...
Found  1465  valid sentences
Found  894  sentences, now processing...
Found  846  valid sentences
Found  111  sentences, now processing...
Found  110  valid sentences
Found  391  sentences, now processing...
Found  390  valid sentences
Found  65  sentences, now processing...
Found  65  valid sentences
In total we have 2876 training examples


<h1>Extracting and processing the Test Set</h1>

In [21]:
locations = ["AIMed-answer-key.txt","BioInfer-answer-key.txt","HPRD50-answer-key.txt","IEPA-answer-key.txt","LLL-answer-key.txt"]




#AIMed_rawAnswers = 

BioInfer_rawAnswers = open("Datasets/testAnswers/BioInfer-answer-key.txt", "r").read()

HPRD50_rawAnswers = open("Datasets/testAnswers/HPRD50-answer-key.txt", "r").read()

IEPA_rawAnswers = open("Datasets/testAnswers/IEPA-answer-key.txt", "r").read()

LLL_rawAnswers = open("Datasets/testAnswers/AIMed-answer-key.txt", "r").read()


In [22]:
def extractAnswers(location):

    result = []

    for a in open(location, "r").read().split("\n"):
        result = result + [a.split(" ")]
   
    dicti = {}

    for r in result:
        if r[0]!='':
            #print(r[0],r[1])
            dicti[r[0]]=r[1] 

    return dicti


In [23]:
locations = ["AIMed-answer-key.txt","BioInfer-answer-key.txt","HPRD50-answer-key.txt","IEPA-answer-key.txt","LLL-answer-key.txt"]

path = "Datasets/testAnswers/"

AIMed_Answers = extractAnswers(path+locations[0])

BioInfer_Answers = extractAnswers(path+locations[1])

HPRD50_Answers = extractAnswers(path+locations[2])

IEPA_Answers = extractAnswers(path+locations[3])

LLL_Answers = extractAnswers(path+locations[4])


#for key,value in AIMed_Answers.items():
    #print(key,value)
    
    
testAnswers =dict(AIMed_Answers)

testAnswers.update(BioInfer_Answers)
testAnswers.update(HPRD50_Answers)
testAnswers.update(IEPA_Answers)
testAnswers.update(LLL_Answers)

for key,value in testAnswers.items():
    print(key,value)
    print(type(value))


AIMed.d3.s30.p0 False
<class 'str'>
AIMed.d3.s30.p1 False
<class 'str'>
AIMed.d3.s30.p2 False
<class 'str'>
AIMed.d3.s31.p0 False
<class 'str'>
AIMed.d3.s31.p1 False
<class 'str'>
AIMed.d3.s31.p2 False
<class 'str'>
AIMed.d3.s31.p3 False
<class 'str'>
AIMed.d3.s31.p4 False
<class 'str'>
AIMed.d3.s31.p5 False
<class 'str'>
AIMed.d3.s31.p6 False
<class 'str'>
AIMed.d3.s31.p7 False
<class 'str'>
AIMed.d3.s31.p8 False
<class 'str'>
AIMed.d3.s31.p9 False
<class 'str'>
AIMed.d3.s32.p0 False
<class 'str'>
AIMed.d3.s33.p0 False
<class 'str'>
AIMed.d3.s33.p1 False
<class 'str'>
AIMed.d3.s33.p2 False
<class 'str'>
AIMed.d3.s33.p3 False
<class 'str'>
AIMed.d3.s33.p4 False
<class 'str'>
AIMed.d3.s33.p5 False
<class 'str'>
AIMed.d3.s33.p6 False
<class 'str'>
AIMed.d3.s33.p7 False
<class 'str'>
AIMed.d3.s33.p8 False
<class 'str'>
AIMed.d3.s33.p9 False
<class 'str'>
AIMed.d3.s33.p10 False
<class 'str'>
AIMed.d3.s33.p11 False
<class 'str'>
AIMed.d3.s33.p12 False
<class 'str'>
AIMed.d3.s33.p13 False
<c

AIMed.d31.s266.p8 False
<class 'str'>
AIMed.d31.s266.p9 False
<class 'str'>
AIMed.d31.s266.p10 False
<class 'str'>
AIMed.d31.s266.p11 False
<class 'str'>
AIMed.d31.s266.p12 False
<class 'str'>
AIMed.d31.s266.p13 False
<class 'str'>
AIMed.d31.s266.p14 False
<class 'str'>
AIMed.d31.s267.p0 False
<class 'str'>
AIMed.d31.s267.p1 False
<class 'str'>
AIMed.d31.s267.p2 False
<class 'str'>
AIMed.d34.s285.p0 False
<class 'str'>
AIMed.d34.s288.p0 False
<class 'str'>
AIMed.d34.s288.p1 True
<class 'str'>
AIMed.d34.s288.p2 True
<class 'str'>
AIMed.d34.s289.p0 True
<class 'str'>
AIMed.d38.s320.p0 True
<class 'str'>
AIMed.d38.s321.p0 False
<class 'str'>
AIMed.d38.s321.p1 False
<class 'str'>
AIMed.d38.s321.p2 False
<class 'str'>
AIMed.d38.s323.p0 False
<class 'str'>
AIMed.d38.s323.p1 False
<class 'str'>
AIMed.d38.s323.p2 False
<class 'str'>
AIMed.d38.s323.p3 False
<class 'str'>
AIMed.d38.s323.p4 False
<class 'str'>
AIMed.d38.s323.p5 False
<class 'str'>
AIMed.d38.s324.p0 False
<class 'str'>
AIMed.d38.s

<class 'str'>
AIMed.d221.s1922.p4 False
<class 'str'>
AIMed.d221.s1922.p5 False
<class 'str'>
AIMed.d221.s1924.p0 False
<class 'str'>
AIMed.d221.s1925.p0 False
<class 'str'>
AIMed.d223.s1934.p0 True
<class 'str'>
AIMed.d223.s1935.p0 False
<class 'str'>
AIMed.d223.s1936.p0 True
<class 'str'>
AIMed.d223.s1938.p0 False
<class 'str'>
AIMed.d223.s1938.p1 True
<class 'str'>
AIMed.d223.s1938.p2 True
<class 'str'>
AIMed.d223.s1938.p3 False
<class 'str'>
AIMed.d223.s1938.p4 False
<class 'str'>
AIMed.d223.s1938.p5 False
<class 'str'>
AIMed.d223.s1939.p0 False
<class 'str'>
AIMed.d223.s1940.p0 False
<class 'str'>
AIMed.d223.s1941.p0 True
<class 'str'>
AIMed.d223.s1942.p0 False
<class 'str'>
AIMed.d223.s1943.p0 False
<class 'str'>
AIMed.d223.s1943.p1 False
<class 'str'>
AIMed.d223.s1943.p2 False
<class 'str'>
AIMed.d223.s1944.p0 True
<class 'str'>
BioInfer.d13.s0.p0 True
<class 'str'>
BioInfer.d13.s0.p1 True
<class 'str'>
BioInfer.d13.s0.p2 True
<class 'str'>
BioInfer.d13.s0.p3 False
<class 'str'>

BioInfer.d503.s0.p4 False
<class 'str'>
BioInfer.d503.s0.p5 False
<class 'str'>
BioInfer.d504.s0.p0 False
<class 'str'>
BioInfer.d508.s0.p0 True
<class 'str'>
BioInfer.d511.s0.p0 False
<class 'str'>
BioInfer.d513.s0.p0 False
<class 'str'>
BioInfer.d513.s0.p1 False
<class 'str'>
BioInfer.d513.s0.p2 False
<class 'str'>
BioInfer.d516.s0.p0 True
<class 'str'>
BioInfer.d516.s1.p0 False
<class 'str'>
BioInfer.d516.s1.p1 False
<class 'str'>
BioInfer.d516.s1.p2 False
<class 'str'>
BioInfer.d516.s1.p3 False
<class 'str'>
BioInfer.d516.s1.p4 False
<class 'str'>
BioInfer.d516.s1.p5 False
<class 'str'>
BioInfer.d516.s1.p6 False
<class 'str'>
BioInfer.d516.s1.p7 False
<class 'str'>
BioInfer.d516.s1.p8 False
<class 'str'>
BioInfer.d516.s1.p9 False
<class 'str'>
BioInfer.d516.s1.p10 False
<class 'str'>
BioInfer.d516.s1.p11 False
<class 'str'>
BioInfer.d516.s1.p12 False
<class 'str'>
BioInfer.d516.s1.p13 False
<class 'str'>
BioInfer.d516.s1.p14 True
<class 'str'>
BioInfer.d518.s0.p0 True
<class 'str'>

<class 'str'>
LLL.d34.s0.p0 True
<class 'str'>
LLL.d34.s0.p1 True
<class 'str'>
LLL.d34.s0.p2 True
<class 'str'>
LLL.d35.s0.p0 False
<class 'str'>
LLL.d35.s0.p1 True
<class 'str'>
LLL.d35.s0.p2 True
<class 'str'>
LLL.d35.s1.p0 False
<class 'str'>
LLL.d35.s1.p1 False
<class 'str'>
LLL.d35.s1.p2 True
<class 'str'>
LLL.d35.s1.p3 False
<class 'str'>
LLL.d35.s1.p4 True
<class 'str'>
LLL.d35.s1.p5 True
<class 'str'>
LLL.d37.s0.p0 False
<class 'str'>
LLL.d37.s0.p1 True
<class 'str'>
LLL.d37.s0.p2 True
<class 'str'>
LLL.d38.s0.p0 True
<class 'str'>
LLL.d38.s0.p1 True
<class 'str'>
LLL.d38.s0.p2 False
<class 'str'>
LLL.d43.s0.p0 False
<class 'str'>
LLL.d43.s0.p1 False
<class 'str'>
LLL.d43.s0.p2 True
<class 'str'>
LLL.d43.s0.p3 False
<class 'str'>
LLL.d43.s0.p4 True
<class 'str'>
LLL.d43.s0.p5 False
<class 'str'>
LLL.d43.s0.p6 False
<class 'str'>
LLL.d43.s0.p7 False
<class 'str'>
LLL.d43.s0.p8 True
<class 'str'>
LLL.d43.s0.p9 False
<class 'str'>
LLL.d43.s0.p10 True
<class 'str'>
LLL.d43.s0.p11 

In [24]:
def extractAndProcessTest(location):
    root = ET.parse(location).getroot()
    sentences = []
    
    for document in root:
        for sentence in document:
            currentSentence = Sentence()
            #print(sentence.attrib["text"])
            currentSentence.text = sentence.attrib["text"]

            for stuff in sentence:
                if stuff.tag == "entity":
                    #print(stuff.attrib["id"])
                    currentSentence.proteins = currentSentence.proteins + [stuff.attrib["id"]]

                if stuff.tag == "pair":
                    #print(stuff.attrib["e1"],stuff.attrib["e2"],stuff.attrib["interaction"])
                    currentSentence.interactions = currentSentence.interactions + [[stuff.attrib["e1"],stuff.attrib["e2"],stuff.attrib["id"]]]
            #print()
            sentences = sentences + [currentSentence]

    #originals = copy.deepcopy(sentences)        

    '''
    for s in sentences:
        s.show()
    '''    
    print("Found ",len(sentences)," sentences, now processing...")
    
    for s in sentences:
        s.fullProcess()
        
    valids = [sentence for sentence in sentences if sentence.valid]
    
    print("Found ",len(valids)," valid sentences")
    
    return valids




In [25]:
locations = ['AIMed-test.xml','BioInfer-test.xml','HPRD50-test.xml','IEPA-test.xml','LLL-test.xml']

testSet = []

for l in locations:
    testSet = testSet + extractAndProcessTest("Datasets/test/"+l)
    
print("In total we have",len(testSet),"test examples")

Found  389  sentences, now processing...
Found  356  valid sentences
Found  206  sentences, now processing...
Found  193  valid sentences
Found  34  sentences, now processing...
Found  34  valid sentences
Found  95  sentences, now processing...
Found  95  valid sentences
Found  12  sentences, now processing...
Found  12  valid sentences
In total we have 690 test examples


In [26]:
#convert interactions to false/true

for t in testSet:
    if len(t.interactions):
        for i in t.interactions:
            print(i[2])
            i[2]=testAnswers[i[2]]
            print(i[2])
        print()

AIMed.d3.s31.p0
False
AIMed.d3.s31.p1
False
AIMed.d3.s31.p2
False
AIMed.d3.s31.p3
False
AIMed.d3.s31.p4
False
AIMed.d3.s31.p5
False
AIMed.d3.s31.p6
False
AIMed.d3.s31.p7
False
AIMed.d3.s31.p8
False
AIMed.d3.s31.p9
False

AIMed.d3.s32.p0
False

AIMed.d3.s33.p0
False
AIMed.d3.s33.p1
False
AIMed.d3.s33.p2
False
AIMed.d3.s33.p3
False
AIMed.d3.s33.p4
False
AIMed.d3.s33.p5
False
AIMed.d3.s33.p6
False
AIMed.d3.s33.p7
False
AIMed.d3.s33.p8
False
AIMed.d3.s33.p9
False
AIMed.d3.s33.p10
False
AIMed.d3.s33.p11
False
AIMed.d3.s33.p12
False
AIMed.d3.s33.p13
False
AIMed.d3.s33.p14
False

AIMed.d3.s34.p0
False
AIMed.d3.s34.p1
False
AIMed.d3.s34.p2
False
AIMed.d3.s34.p3
False
AIMed.d3.s34.p4
False
AIMed.d3.s34.p5
False
AIMed.d3.s34.p6
False
AIMed.d3.s34.p7
False
AIMed.d3.s34.p8
False
AIMed.d3.s34.p9
False

AIMed.d3.s35.p0
False
AIMed.d3.s35.p1
False
AIMed.d3.s35.p2
False
AIMed.d3.s35.p3
False
AIMed.d3.s35.p4
False
AIMed.d3.s35.p5
False
AIMed.d3.s35.p6
False
AIMed.d3.s35.p7
False
AIMed.d3.s35.p8
False
A

False
BioInfer.d74.s0.p3
False
BioInfer.d74.s0.p4
True
BioInfer.d74.s0.p5
False
BioInfer.d74.s0.p6
False
BioInfer.d74.s0.p7
True
BioInfer.d74.s0.p8
False
BioInfer.d74.s0.p9
False
BioInfer.d74.s0.p10
False
BioInfer.d74.s0.p11
False
BioInfer.d74.s0.p12
False
BioInfer.d74.s0.p13
False
BioInfer.d74.s0.p14
False
BioInfer.d74.s0.p15
True
BioInfer.d74.s0.p16
False
BioInfer.d74.s0.p17
False
BioInfer.d74.s0.p18
True
BioInfer.d74.s0.p19
False
BioInfer.d74.s0.p20
True
BioInfer.d74.s0.p21
False
BioInfer.d74.s0.p22
False
BioInfer.d74.s0.p23
True
BioInfer.d74.s0.p24
True
BioInfer.d74.s0.p25
False
BioInfer.d74.s0.p26
False
BioInfer.d74.s0.p27
True

BioInfer.d80.s0.p0
True
BioInfer.d80.s0.p1
False
BioInfer.d80.s0.p2
False
BioInfer.d80.s0.p3
True
BioInfer.d80.s0.p4
False
BioInfer.d80.s0.p5
False
BioInfer.d80.s0.p6
False
BioInfer.d80.s0.p7
False
BioInfer.d80.s0.p8
False
BioInfer.d80.s0.p9
False
BioInfer.d80.s0.p10
False
BioInfer.d80.s0.p11
False
BioInfer.d80.s0.p12
True
BioInfer.d80.s0.p13
False
BioInfe

False
BioInfer.d422.s0.p29
False
BioInfer.d422.s0.p30
False
BioInfer.d422.s0.p31
False
BioInfer.d422.s0.p32
False
BioInfer.d422.s0.p33
False
BioInfer.d422.s0.p34
False
BioInfer.d422.s0.p35
False
BioInfer.d422.s0.p36
False
BioInfer.d422.s0.p37
False
BioInfer.d422.s0.p38
False
BioInfer.d422.s0.p39
False
BioInfer.d422.s0.p40
False
BioInfer.d422.s0.p41
False
BioInfer.d422.s0.p42
False
BioInfer.d422.s0.p43
False
BioInfer.d422.s0.p44
False
BioInfer.d422.s0.p45
False
BioInfer.d422.s0.p46
False
BioInfer.d422.s0.p47
False
BioInfer.d422.s0.p48
False
BioInfer.d422.s0.p49
False
BioInfer.d422.s0.p50
False
BioInfer.d422.s0.p51
False
BioInfer.d422.s0.p52
False
BioInfer.d422.s0.p53
False
BioInfer.d422.s0.p54
False

BioInfer.d424.s0.p0
False
BioInfer.d424.s0.p1
False
BioInfer.d424.s0.p2
False
BioInfer.d424.s0.p3
False
BioInfer.d424.s0.p4
False
BioInfer.d424.s0.p5
False
BioInfer.d424.s0.p6
False
BioInfer.d424.s0.p7
False
BioInfer.d424.s0.p8
False
BioInfer.d424.s0.p9
False
BioInfer.d424.s0.p10
False
BioI

False
BioInfer.d811.s0.p2
False

BioInfer.d816.s0.p0
True

BioInfer.d818.s0.p0
False
BioInfer.d818.s0.p1
True
BioInfer.d818.s0.p2
False

BioInfer.d820.s0.p0
False
BioInfer.d820.s0.p1
False
BioInfer.d820.s0.p2
False
BioInfer.d820.s0.p3
False
BioInfer.d820.s0.p4
False
BioInfer.d820.s0.p5
True

BioInfer.d824.s0.p0
True

BioInfer.d825.s0.p0
True

HPRD50.d6.s0.p0
False

HPRD50.d6.s1.p0
False

HPRD50.d6.s2.p0
False
HPRD50.d6.s2.p1
False
HPRD50.d6.s2.p2
False

HPRD50.d6.s3.p0
False
HPRD50.d6.s3.p1
False
HPRD50.d6.s3.p2
False

HPRD50.d6.s4.p0
True

HPRD50.d10.s0.p0
True
HPRD50.d10.s0.p1
True
HPRD50.d10.s0.p2
False

HPRD50.d10.s1.p0
False
HPRD50.d10.s1.p1
False
HPRD50.d10.s1.p2
False

HPRD50.d10.s2.p0
True

HPRD50.d10.s3.p0
True
HPRD50.d10.s3.p1
False
HPRD50.d10.s3.p2
False

HPRD50.d10.s4.p0
True
HPRD50.d10.s4.p1
False
HPRD50.d10.s4.p2
False

HPRD50.d15.s0.p0
True

HPRD50.d15.s1.p0
True

HPRD50.d15.s2.p0
True
HPRD50.d15.s2.p1
False
HPRD50.d15.s2.p2
False

HPRD50.d15.s3.p0
True

HPRD50.d15.s4.p0

<h1>Select relevant examples with actual interactions</h1>

In [27]:
print(len(trainSet))
print(len(testSet))

trainSet = [t for t in trainSet if len(t.interactions)]


testSet = [t for t in testSet if len(t.interactions)]

print()


trainSet = [t for t in trainSet if len(t.words)<=100]

testSet = [t for t in testSet if len(t.words)<=100]

print(len(trainSet))
print(len(testSet))

2876
690

2245
529


<h1>Exporting the processed Dataset</h1>

In [28]:
with open('datasetTrain.pkl', 'wb') as output:
    pickle.dump(trainSet, output, pickle.HIGHEST_PROTOCOL)
    
with open('datasetTest.pkl', 'wb') as output:    
    pickle.dump(testSet, output, pickle.HIGHEST_PROTOCOL)

print("saved")

saved


In [29]:

try:
    del trainSet
    del testSet
except NameError:
    print("nope")

try:
    print(len(trainSet))
    print(len(testSet))
except NameError:
    print("nope")

nope


In [30]:
with open('datasetTrain.pkl', 'rb') as input1:
    trainSet = pickle.load(input1)
    
with open('datasetTest.pkl', 'rb') as input2:    
    testSet = pickle.load(input2)
    
try:
    print(len(trainSet))
    print(len(testSet))
except NameError:
    print("nope")   

2245
529


In [31]:
print(len(trainSet))
print(trainSet[0].words)
print(trainSet[0].text)
print(trainSet[0].interactions)

2245
['Cytokines', 'measurements', 'during', 'PROTEIN0', 'treatment', 'showed', 'a', 'trend', 'to', 'decreasing', 'levels', 'of', 'PROTEIN1', 'at', '4,', '12,', 'and', '24', 'weeks.']
Cytokines measurements during _________ treatment showed a trend to decreasing levels of ____ at 4, 12, and 24 weeks.
[['PROTEIN0', 'PROTEIN1', 'False']]
